# Bitcoin Macro Analysis Visualized

In [2]:

import pandas as pd
import numpy as np
import statsmodels.api as sm


In [3]:

# Load BTC Data
btc_control = pd.read_csv("../data/btc_control_period_clean.csv", parse_dates=["date"])
btc_event = pd.read_csv("../data/btc_event_period_clean.csv", parse_dates=["date"])

btc_control['log_return'] = np.log(btc_control['close'] / btc_control['close'].shift(1))
btc_event['log_return'] = np.log(btc_event['close'] / btc_event['close'].shift(1))

btc_all = pd.concat([btc_control, btc_event])
btc_all['event_period'] = btc_all['date'].apply(lambda d: 1 if d in btc_event['date'].values else 0)


In [4]:

# Load SP500 and VIX data
sp500_control = pd.read_csv("../data/sp500_control_period_clean.csv", parse_dates=["date"])
sp500_event = pd.read_csv("../data/sp500_event_period_clean.csv", parse_dates=["date"])
sp500_all = pd.concat([sp500_control, sp500_event])
sp500_all.rename(columns={"close": "sp500_close"}, inplace=True)
sp500_all['sp500_return'] = sp500_all['sp500_close'].pct_change()

vix_control = pd.read_csv("../data/vix_control_period_clean.csv", parse_dates=["date"])
vix_event = pd.read_csv("../data/vix_event_period_clean.csv", parse_dates=["date"])
vix_all = pd.concat([vix_control, vix_event])
vix_all.rename(columns={"close": "vix_close"}, inplace=True)
vix_all['vix_change'] = vix_all['vix_close'].pct_change()


In [5]:

# Load Treasury yield data
treasury_control = pd.read_csv("../data/treasury_yield_control_period_clean.csv", parse_dates=["date"])
treasury_event = pd.read_csv("../data/treasury_yield_event_period_clean.csv", parse_dates=["date"])
treasury_all = pd.concat([treasury_control, treasury_event])
treasury_all['yield_change'] = treasury_all['treasury_yield'].diff()


In [6]:

# Merge into final dataset
btc_all = btc_all.merge(sp500_all[['date', 'sp500_return']], on='date', how='left')
btc_all = btc_all.merge(vix_all[['date', 'vix_change']], on='date', how='left')
btc_all = btc_all.merge(treasury_all[['date', 'yield_change']], on='date', how='left')
btc_all.dropna(subset=['log_return', 'sp500_return', 'vix_change', 'yield_change'], inplace=True)


In [7]:

# Run Difference-in-Differences regression
X = sm.add_constant(btc_all[['event_period', 'sp500_return', 'vix_change', 'yield_change']])
y = btc_all['log_return']

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:             log_return   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     4.165
Date:                Fri, 21 Mar 2025   Prob (F-statistic):            0.00664
Time:                        00:18:48   Log-Likelihood:                 102.37
No. Observations:                  44   AIC:                            -194.7
Df Residuals:                      39   BIC:                            -185.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0015      0.005      0.283   